In [43]:
from ml.data.datasets import StarWars

dataset = StarWars()
data = dataset[0]
data

[2022-02-08 20:52:45,304][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-08 20:52:45,304][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-08 20:52:45,305][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-08 20:52:45,305][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-08 20:52:45,306][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-08 20:52:45,306][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVersion'>, drop extra fields: True
[2022-02-08 20:52:45,307][simple_parsing.helpers.serialization.decoding][DEBUG] name = type, field_type = <enum 'DatasetVersi

HeteroData(
  Character={ x=[113, 32] },
  (Character, INTERACTIONS, Character)={
    edge_attr=[958, 0],
    edge_index=[2, 958],
    timestamp=[958]
  },
  (Character, MENTIONS, Character)={
    edge_attr=[1120, 0],
    edge_index=[2, 1120],
    timestamp=[1120]
  }
)

In [44]:
from ml.data import EdgeLoaderDataModule

data_module = EdgeLoaderDataModule(data, batch_size=16, num_samples=[4] * 2, num_workers=4, node_type='Character')

In [45]:
from typing import Optional

import torch
import torch.nn.functional as F
from torch_geometric.typing import Metadata
from torch_geometric.nn import HGTConv, Linear
import torchmetrics
import pytorch_lightning as pl

In [46]:
class HGTModule(torch.nn.Module):
    def __init__(
            self,
            node_type,
            metadata: Metadata,
            hidden_channels=64,
            num_heads=2,
            num_layers=1
    ):
        super().__init__()
        self.node_type = node_type
        self.lin_dict = torch.nn.ModuleDict()
        for node_type in metadata[0]:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, metadata, num_heads, group='sum')
            self.convs.append(conv)

    def forward(self, x_dict, edge_index_dict):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return x_dict[self.node_type]


class ClusterModule(torch.nn.Module):
    def __init__(
            self,
            n_clusters: int = 5,
            embedding_dim: int = 64,
            alpha: float = 1.0,
            cluster_centers: Optional[torch.Tensor] = None,
    ) -> None:
        super().__init__()
        self.n_clusters = n_clusters
        self.embedding_dim = embedding_dim
        self.alpha = alpha

        if cluster_centers is None:
            initial_cluster_centers = torch.zeros(
                self.n_clusters, self.embedding_dim, dtype=torch.float
            )
            torch.nn.init.xavier_uniform_(initial_cluster_centers)
        else:
            assert cluster_centers.shape == (self.n_clusters, self.embedding_dim)
            initial_cluster_centers = cluster_centers
        self.cluster_centers = torch.nn.Parameter(initial_cluster_centers)

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        # Compute soft assignments (returns t distribution for each node over clusters)
        norm_squared = torch.sum(torch.square(batch.unsqueeze(1) - self.cluster_centers), dim=2)
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2
        numerator = torch.pow(numerator, power)
        return numerator / torch.sum(numerator, dim=1, keepdim=True)

In [47]:
class Net(pl.LightningModule):
    def __init__(
            self,
            node_type,
            metadata: Metadata,
            n_clusters: int = 5,
            embedding_dim=64,
            num_heads=2,
            num_layers=1
    ):
        super().__init__()
        self.node_type = node_type
        self.node_embedding = HGTModule(node_type, metadata, embedding_dim, num_heads, num_layers)
        self.cluster_embedding = ClusterModule(n_clusters, embedding_dim)

        self.dist = torch.nn.PairwiseDistance(p=2)
        self.lin = torch.nn.Linear(1, 2)
        self.ce_loss = torch.nn.CrossEntropyLoss()

        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()


    def _forward_step_small(self, batch, mode='train'):
        self.cluster_embedding.requires_grad_(False)

        # Small Step: Link prediction
        batch_l, batch_r, label = batch
        batch_size = batch_l[self.node_type].batch_size

        emb_l = self.node_embedding(batch_l.x_dict, batch_l.edge_index_dict)[:batch_size]
        emb_r = self.node_embedding(batch_r.x_dict, batch_r.edge_index_dict)[:batch_size]

        # Compute homophily based loss. TODO: do we need to cap the individual losses?
        dist = self.dist(emb_l, emb_r)
        # hp_loss = torch.mean(dist * (label * 2.0 - 1.0))
        out = self.lin(torch.unsqueeze(dist, 1))
        hp_loss = self.ce_loss(out, label)

        loss = hp_loss


        pred = out.argmax(dim=-1)
        if mode == 'train':
            self.train_acc.update(pred, label)
        elif mode == 'val':
            self.val_acc.update(pred, label)

        return loss

    def _forward_step_large(self, batch):
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch):
        loss = self._forward_step_small(batch, mode='train')
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._forward_step_small(batch, mode='val')
        return loss

    def training_epoch_end(self, outputs) -> None:
        acc = self.train_acc.compute()
        self.log('mean_train_acc', acc, prog_bar=True)
        self.train_acc.reset()

    def validation_epoch_end(self, outputs) -> None:
        acc = self.val_acc.compute()
        self.log('mean_val_acc', acc, prog_bar=True)
        self.val_acc.reset()

    def forward(self, batch):
        batch_size = batch[self.node_type].batch_size
        emb = self.node_embedding(batch.x_dict, batch.edge_index_dict)[:batch_size]
        return emb


model = Net(node_type='Character', metadata=data.metadata(), n_clusters=5, embedding_dim=64, num_heads=2, num_layers=1)
trainer = pl.Trainer(
    gpus=1,
    callbacks=[
        pl.callbacks.EarlyStopping(monitor="mean_val_acc", min_delta=0.00, patience=8, verbose=True, mode="max")
    ],
    max_epochs=50,
)
trainer.fit(model, data_module)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [48]:
predictions = trainer.predict(model, data_module)
embeddings = torch.cat(predictions, dim=0).cpu().numpy()

Predicting: 182it [00:00, ?it/s]

# Extract and cluster the embeddings

In [49]:
from shared.constants import BENCHMARKS_RESULTS

save_path = BENCHMARKS_RESULTS.joinpath('analysis', 'pyg-hgt-homophily-clustering')
save_path.mkdir(parents=True, exist_ok=True)

In [50]:
import faiss
import pandas as pd

In [51]:
k = 5
kmeans = faiss.Kmeans(embeddings.shape[1], k, niter=20, verbose=True, nredo=5)
kmeans.train(embeddings)

Clustering 113 points in 64D to 5 clusters, redo 5 times, 20 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 19 (0.00 s, search 0.00 s): objective=368.352 imbalance=1.262 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 19 (0.01 s, search 0.01 s): objective=370.535 imbalance=1.088 nsplit=0       
Outer iteration 2 / 5
  Iteration 19 (0.01 s, search 0.01 s): objective=368.352 imbalance=1.262 nsplit=0       
Outer iteration 3 / 5
  Iteration 19 (0.01 s, search 0.01 s): objective=390.622 imbalance=1.091 nsplit=0       
Outer iteration 4 / 5


WARNING clustering 113 points to 5 centroids: please provide at least 195 training points


368.35211181640625

In [52]:
D, I = kmeans.index.search(embeddings, 1)

In [53]:
from shared.graph import CommunityAssignment

labeling = pd.Series(I.squeeze(), index=dataset.node_mapping(), name="cid")
labeling.index.name = "nid"
comlist = CommunityAssignment(labeling)

In [54]:
comlist.save_comlist(save_path.joinpath('schema.comlist'))

In [ ]:
from datasets.scripts import export_to_visualization

export_to_visualization.run(
    export_to_visualization.Args(
        dataset='star-wars',
        version='base',
        run_paths=[str(save_path)]
    )
)

[2022-02-08 20:58:35,175][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-08 20:58:35,177][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-08 20:58:35,178][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-08 20:58:35,179][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-08 20:58:35,180][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-08 20:58:35,180][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVersion'>, drop extra fields: True
[2022-02-08 20:58:35,181][simple_parsing.helpers.serialization.decoding][DEBUG] name = type, field_type = <enum 'DatasetVersi

# Calculate Evaluation Metrics

In [55]:
from shared.graph import DataGraph
from benchmarks.evaluation import get_metric_list

In [56]:
from shared.schema import GraphSchema, DatasetSchema

DATASET = DatasetSchema.load_schema('star-wars')
schema = GraphSchema.from_dataset(DATASET)
G = DataGraph.from_schema(schema)

ERROR! Session/line number was not unique in database. History logging moved to new session 102
[2022-02-08 20:58:35,830][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-08 20:58:35,831][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-08 20:58:35,832][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-08 20:58:35,833][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-08 20:58:35,833][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-08 20:58:35,834][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVersion'>, drop extra fields: True
[2022-02-08 20:58:35,835][sim

In [57]:
metrics = get_metric_list(ground_truth=False, overlapping=False)

results = pd.DataFrame([
    {
        'metric': metric_cls.metric_name(),
        'value': metric_cls.calculate(G, comlist)
    }
    for metric_cls in metrics]
)
results

,metric,value
0,community_count,NaN
1,conductance,0.439275
2,expansion,3.807287
3,internal_edge_density,0.361212
4,avg_odf,7.617955
5,modularity_overlap,0.062602
6,link_modularity,0.043567
7,z_modularity,0.614605
8,modularity,-0.015605
